11.2.4. Neumann Laplace Direct Method 
======================================================
**keys**: homogeneous Neumann bvp, hypersingular operator, unknown Dirichlet data

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve.solvers import CG

Consider the Neumann boundary value problem 

$$ \left\{ \begin{array}{rlc l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_1 u&=& u_1, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

The unique solution $u\in H^1(\Omega)$ can be written in the following form (representation forumula) 

$$ u(x) = \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, u_1(y)\, \mathrm{d}\sigma_y}_{\displaystyle{ \mathrm{SL}(u_1) }} + \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{\langle n(y), x-y \rangle}{\| x-y\|^3} } \, u_1( y)\, \mathrm{d}\sigma_y}_{\displaystyle{ \mathrm{DL}(u_1) }}\,.$$ 

Let's carefully apply the Neumann trace $\gamma_1$ to the representation formula and solve the resulting boundary integral equation for the Dirichlet trace $u_0$ of $u$ by discretisation of the following variational formulation: 

$$ \forall \, v\in H^{\frac12}(\Gamma):  \left\langle v, \gamma_1 \left(\mathrm{DL}(u_0)\right) \right\rangle_{-\frac12}  = \left\langle u_1, v\right\rangle_{-\frac12} - \left\langle v, \gamma_1 \left(\mathrm{SL}(u_1)\right) \right\rangle_{-\frac12}\,. $$

Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Glue(Sphere( (0,0,0), 1).faces)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.2)).Curve(2)

Define conforming finite element spaces for $H^{\frac12}(\Gamma)$ and $H^{-\frac12}(\Gamma)$ respectively:  

In [ ]:
fesL2 = SurfaceL2(mesh, order=2, dual_mapping=False) 
u,v = fesL2.TnT()
fesH1 = H1(mesh, order=2, definedon=mesh.Boundaries(".*"))
uH1,vH1 = fesH1.TnT()
print ("ndofL2 = ", fesL2.ndof, "ndof H1 = ", fesH1.ndof)

Project the given Neumann data $u_1$ in finite element space and have a look at the boundary data:

In [ ]:
uexa = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )

graduexa = CF( (uexa.Diff(x), uexa.Diff(y), uexa.Diff(z)) )
n = specialcf.normal(3)
u1 = GridFunction(fesL2)
u1.Interpolate(graduexa*n, definedon=mesh.Boundaries(".*"))

Draw (u1, mesh, draw_vol=False, order=3);

The discretisation of the variational formulation leads to a system of linear equations, ie 

$$ \left( \mathrm{D} + \mathrm{S}\right) \, \mathrm{u}_0 = \left( \frac12 \,\mathrm{M} - \mathrm{K}' \right) \, \mathrm{u}_1\,, $$

where the linear operators are as follows
- $\mathrm{D}$ is the hypersingular operator and the stabilisation $(\mathrm D + \mathrm{S})$ is regular and symmetric.
- $\mathrm{M}$ is a mass matrix.
- $\mathrm{K}'$ is the adjoint double layer operator. 

The stabilisation matrix $S$ is needed to cope with the kernel of the hypersingular operator. The following stabilisation matrix is used here: 

$$ S \in \mathbb R^{n\times n}, \quad S_{ij} = \int\limits_{\Gamma} v_i(x) \,\mathrm{d} \sigma \cdot \int\limits_{\Gamma} v_j(x) \,\mathrm{d} \sigma\,,$$ 

where $v_i, v_j$ being $H^{\frac12}(\Gamma)$-conforming shape functions. 

In [ ]:
vH1m1 = LinearForm(vH1*1*ds(bonus_intorder=3)).Assemble()
S = (BaseMatrix(Matrix(vH1m1.vec.Reshape(1))))@(BaseMatrix(Matrix(vH1m1.vec.Reshape(fesH1.ndof))))

Now we assemble the stabilised system matrix and the right hand side and solve for the Dirichlet data $u_0$ with an iterative solver:

In [ ]:
u0 = GridFunction(fesH1)
pre = BilinearForm(uH1*vH1*ds).Assemble().mat.Inverse(freedofs=fesH1.FreeDofs()) 
with TaskManager():
    D = HypersingularOperator(fesH1, intorder=12)
    
    M = BilinearForm( v.Trace() * uH1 * ds(bonus_intorder=3)).Assemble()
    Kt = DoubleLayerPotentialOperator(fesL2, fesH1, intorder=12)    
    rhs = ( (0.5 * M.mat.T - Kt.mat) * u1.vec ).Evaluate()

    CG(mat=D.mat+S, pre=pre, rhs=rhs, sol=u0.vec, tol=1e-8, maxsteps=200, printrates=True)

Let's have a look at the numerical and exact Dirichlet data and compare them quantitatively:

In [ ]:
Draw (u0, mesh, draw_vol=False, order=3);
u0exa = GridFunction(fesH1)
u0exa.Interpolate (uexa, definedon=mesh.Boundaries(".*"))
Draw (u0exa, mesh, draw_vol=False, order=3);

In [ ]:
print ("L2 error of surface gradients =", sqrt (Integrate ( (grad(u0exa)-grad(u0))**2, mesh, BND)))